In [ ]:
### converting into datetime format 
from datetime import datetime 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
### mounting the drive 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#data = pd.read_csv('/content/drive/MyDrive/Medium Articles/Indian president data/India president data.csv',encoding='windows-1254')
columns = ['Name', 'sex', 'dob', 'dod', 'Service_start',
       'Service_End ', 'Election', 'Party', 'Religion', 'Education ',
       'Highest edcation', 'Birthplace', 'profession',
       'Civilian Awards & Honours', 'Important points']

data = pd.read_excel('/content/drive/MyDrive/Medium Articles/Indian president data/President_data_final.xlsx',usecols = columns)

In [ ]:
data.head(3)

,Name,sex,dob,dod,Service_start,Service_End,Election,Party,Religion,Education,Highest edcation,Birthplace,profession,Civilian Awards & Honours,Important points
0,Dr. Rajendra Prasad,M,1884-03-12,1963-02-28,1950-01-26,1962-05-13,"[1952, 1957]",INC,Hinduism,"[M.A , LL.D]",LL.D,Bihar,"[Politician, Lawyer, Journalist ]",[Bharat Ratna],1. First president of the Repblic of India.\...
1,Dr. Sarvepalli Radhakrishnan,M,1888-05-09,1975-04-17,1962-05-13,1967-05-13,1962,Independent,Hinduism,"[M.A, D. Litt. (Hony.), LL.D., D.C.L, Litt. D....",LL.D,Tamil NaduÂ,"[Philosopher, academic]",[Bharat Ratna],The man whose birthday is observed and celebra...
2,Dr. Zakir Husain,M,1897-08-02,1969-05-03,1967-05-13,1969-05-03,1967,Independent,Islam,"[M.A, Ph.D]",Ph. D,HyderabadÂ,"[Politician , Economist]","[Bharat Ratna , Padma Vibhushan]",1. First Muslim President of India.\n2. Firs...


In [ ]:
#### creating a copy of data for further operations 
df = data.copy()

In [ ]:
df.rename(columns = {'Service_End ':'Service_End'},inplace= True)

In [ ]:
### converting date columns into datetime format 
df['dob'] = pd.to_datetime(df['dob'])
df['dod'] = pd.to_datetime(df['dod'])
df['Service_start'] = pd.to_datetime(df['Service_start'])
df['Service_End '] = pd.to_datetime(df['Service_End'])

In [ ]:
### There are three acting presidents in the list. filling their missing value in the party colummn with 'acting president'
df. Party = df.Party.fillna('Acting_President')

In [ ]:
### higligting the acting presidents with different colour
df.style.apply(lambda x: ['background:lightgreen' if x.name in [3,4,7] 
                               else '' for i in x], axis=1)

,Name,sex,dob,dod,Service_start,Service_End,Election,Party,Religion,Education,Highest edcation,Birthplace,profession,Civilian Awards & Honours,Important points,Service_End
0,Dr. Rajendra Prasad,M,1884-03-12 00:00:00,1963-02-28 00:00:00,1950-01-26 00:00:00,1962-05-13 00:00:00,"[1952, 1957]",INC,Hinduism,"[M.A , LL.D]",LL.D,Bihar,"[Politician, Lawyer, Journalist ]",[Bharat Ratna],1. First president of the Repblic of India. 2. Only person to have held office for two terms. 3. Indian Independence Movement was led by him.,1962-05-13 00:00:00
1,Dr. Sarvepalli Radhakrishnan,M,1888-05-09 00:00:00,1975-04-17 00:00:00,1962-05-13 00:00:00,1967-05-13 00:00:00,1962,Independent,Hinduism,"[M.A, D. Litt. (Hony.), LL.D., D.C.L, Litt. D., D.L, F.R.S.L, F.B.A., Hony. Fellow, All Souls College (Oxford)]",LL.D,Tamil NaduÂ,"[Philosopher, academic]",[Bharat Ratna],The man whose birthday is observed and celebrated as the Teachers’ Day.,1967-05-13 00:00:00
2,Dr. Zakir Husain,M,1897-08-02 00:00:00,1969-05-03 00:00:00,1967-05-13 00:00:00,1969-05-03 00:00:00,1967,Independent,Islam,"[M.A, Ph.D]",Ph. D,HyderabadÂ,"[Politician , Economist]","[Bharat Ratna , Padma Vibhushan]",1. First Muslim President of India. 2. First president who died while Serving.,1969-05-03 00:00:00
3,Shri Varahagiri Venkata Giri(acting),M,1894-10-08 00:00:00,1980-06-24 00:00:00,1969-05-03 00:00:00,1969-07-20 00:00:00,nan,Acting_President,Hinduism,"[LL.D, Barrister-at-Law, D. Litt]",LL.D,Odisha,"[Lawyer ,Activist , Politician ]",[Bharat Ratna],nan,1969-07-20 00:00:00
4,Shri Mohammad Hidayatullah,M,1905-12-17 00:00:00,1992-09-18 00:00:00,1969-07-20 00:00:00,1969-08-24 00:00:00,nan,Acting_President,Islam,"[Barrister-at-Law , M.A, LL.D]",LL.D,Uttar Pradesh,"[Lawyer, Politician , Academician]",nan,nan,1969-08-24 00:00:00
5,Shri Varahagiri Venkata Giri,M,1894-10-08 00:00:00,1980-06-24 00:00:00,1969-08-24 00:00:00,1974-08-24 00:00:00,1969,Independent,Hinduism,"[LL.D, Barrister-at-Law, D. Litt]",LL.D,Odisha,"[Lawyer ,Activist , Politician ]",[Bharat Ratna],nan,1974-08-24 00:00:00
6,Dr. Fakhruddin Ali Ahmed,M,1905-05-13 00:00:00,1977-02-11 00:00:00,1974-08-24 00:00:00,1977-02-11 00:00:00,1974,INC,Islam,"[M.D, practiced Law]",M.D,Delhi,Lawyer,nan,Second president who died while Serving,1977-02-11 00:00:00
7,Shri Basappa Danappa Jatti,M,1912-09-10 00:00:00,2002-06-07 00:00:00,1977-02-11 00:00:00,1977-07-25 00:00:00,nan,Acting_President,Hinduism,"[B.A, LL.B]",LL.B,Karnataka,"[Politician,Lawyer]",nan,He was the Chief minister of Mysore,1977-07-25 00:00:00
8,Shri Neelam Sanjiva Reddy,M,1913-05-19 00:00:00,1996-06-01 00:00:00,1977-07-25 00:00:00,1982-07-25 00:00:00,1977,Janata Party,Hinduism,[LL.D],LL.D,Andhra Pradesh,[Politician],nan,"From the post of Lok Sabha Speaker, he became the president",1982-07-25 00:00:00
9,Giani Zail Singh,M,1916-05-05 00:00:00,1994-12-25 00:00:00,1982-07-25 00:00:00,1987-07-25 00:00:00,1982,INC,Sikh,nan,nan,Panjab,[Politician],nan,He was Chief Minister of Punjab,1987-07-25 00:00:00


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Name                       18 non-null     object        
 1   sex                        18 non-null     object        
 2   dob                        18 non-null     datetime64[ns]
 3   dod                        15 non-null     datetime64[ns]
 4   Service_start              18 non-null     datetime64[ns]
 5   Service_End                17 non-null     datetime64[ns]
 6   Election                   15 non-null     object        
 7   Party                      18 non-null     object        
 8   Religion                   18 non-null     object        
 9   Education                  17 non-null     object        
 10  Highest edcation           17 non-null     object        
 11  Birthplace                 18 non-null     object        
 12  profession

In [ ]:
### Calculating the total office days of each president in days
df['Num_Service_Days'] = (df['Service_End']- df['Service_start']).dt.days.astype('Int64')

### Calculating the total office days in years
from datetime import timedelta
df['Num_Service_years'] = (df['Service_End'] - df['Service_start'])/timedelta(days=365)

### Calculating the total office days in months
df['Num_Ser_Months'] = ((df['Service_End'] - df['Service_start'])/np.timedelta64(1, 'M'))

In [ ]:
df[['Num_Service_Days','Num_Service_years','Num_Ser_Months']].head()

,Num_Service_Days,Num_Service_years,Num_Ser_Months
0,4490,12.301370,147.518430
1,1826,5.002740,59.993018
2,721,1.975342,23.688371
3,78,0.213699,2.562681
4,35,0.095890,1.149921


In [ ]:
### calculating the age when a president started his services
df['Service_start_Age'] = (df['Service_start'].dt.year - df['dob'].dt.year).astype(int)
df.loc[df['Service_start'].dt.month < df['dob'].dt.month, 'Service_start_Age'] = df['Service_start_Age'] - 1
df.loc[(df['Service_start'].dt.month == df['dob'].dt.month) & (df['Service_start'].dt.day < df['dob'].dt.day), 'Service_start_Age'] = df['Service_start_Age'] - 1

In [ ]:
### calculating the retirnment age of each presidents
df['retairment_Age'] = (df['Service_End '].dt.year - df['dob'].dt.year).astype('Int64')
df.loc[df['Service_End'].dt.month < df['dob'].dt.month, 'retairment_Age'] = df['retairment_Age'] - 1
df.loc[(df['Service_End'].dt.month == df['dob'].dt.month) & (df['Service_End'].dt.day < df['dob'].dt.day), 'retairment_Age'] = df['retairment_Age'] - 1

In [ ]:
### calculating both the service started and retaired age in days
df['Service_start_Age_day']  =(df['Service_start'] - df['dob']).dt.days
df['retairment_Age_day'] = ((df['Service_End'] - df['dob']).dt.days).astype('Int64')

In [ ]:
### calculating the each president's age at death
df['age_at_death'] = (df['dod'].dt.year - df['dob'].dt.year).astype('Int64')
df.loc[df['dod'].dt.month < df['dob'].dt.month, 'age_at_death'] = df['age_at_death'] - 1
df.loc[(df['dod'].dt.month == df['dob'].dt.month) & (df['dod'].dt.day < df['dob'].dt.day), 'age_at_death'] = df['age_at_death'] - 1

In [ ]:
### calculating the age at death in days 
df['age_at_death_days'] = ((df['dod'] - df['dob']).dt.days).astype('Int64')

In [ ]:
df.head()

,Name,sex,dob,dod,Service_start,Service_End,Election,Party,Religion,Education,...,Service_End,Num_Service_Days,Num_Service_years,Num_Ser_Months,Service_start_Age,retairment_Age,Service_start_Age_day,retairment_Age_day,age_at_death,age_at_death_days
0,Dr. Rajendra Prasad,M,1884-03-12,1963-02-28,1950-01-26,1962-05-13,"[1952, 1957]",INC,Hinduism,"[M.A , LL.D]",...,1962-05-13,4490,12.301370,147.518430,65,78,24060,28550,78,28841
1,Dr. Sarvepalli Radhakrishnan,M,1888-05-09,1975-04-17,1962-05-13,1967-05-13,1962,Independent,Hinduism,"[M.A, D. Litt. (Hony.), LL.D., D.C.L, Litt. D....",...,1967-05-13,1826,5.002740,59.993018,74,79,27031,28857,86,31753
2,Dr. Zakir Husain,M,1897-08-02,1969-05-03,1967-05-13,1969-05-03,1967,Independent,Islam,"[M.A, Ph.D]",...,1969-05-03,721,1.975342,23.688371,69,71,25485,26206,71,26206
3,Shri Varahagiri Venkata Giri(acting),M,1894-10-08,1980-06-24,1969-05-03,1969-07-20,NaN,Acting_President,Hinduism,"[LL.D, Barrister-at-Law, D. Litt]",...,1969-07-20,78,0.213699,2.562681,74,74,27235,27313,85,31305
4,Shri Mohammad Hidayatullah,M,1905-12-17,1992-09-18,1969-07-20,1969-08-24,NaN,Acting_President,Islam,"[Barrister-at-Law , M.A, LL.D]",...,1969-08-24,35,0.095890,1.149921,63,63,23226,23261,86,31687


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Name                       18 non-null     object        
 1   sex                        18 non-null     object        
 2   dob                        18 non-null     datetime64[ns]
 3   dod                        15 non-null     datetime64[ns]
 4   Service_start              18 non-null     datetime64[ns]
 5   Service_End                17 non-null     datetime64[ns]
 6   Election                   15 non-null     object        
 7   Party                      18 non-null     object        
 8   Religion                   18 non-null     object        
 9   Education                  17 non-null     object        
 10  Highest edcation           17 non-null     object        
 11  Birthplace                 18 non-null     object        
 12  profession

In [ ]:
df.describe()

,Num_Service_Days,Num_Service_years,Num_Ser_Months,Service_start_Age,retairment_Age,Service_start_Age_day,retairment_Age_day,age_at_death,age_at_death_days
count,17.000000,17.000000,17.000000,18.000000,17.000000,18.000000,17.000000,15.000000,15.000000
mean,1557.529412,4.267204,51.172448,70.055556,74.647059,25785.333333,27482.529412,82.866667,30504.466667
std,1029.921970,2.821704,33.837967,4.746171,5.722119,1764.770140,2079.416689,6.728051,2425.834786
min,35.000000,0.095890,1.149921,63.000000,63.000000,23226.000000,23261.000000,71.000000,26206.000000
25%,902.000000,2.471233,29.635105,65.250000,71.000000,24091.750000,26206.000000,79.500000,29277.500000
50%,1826.000000,5.002740,59.993018,70.500000,76.000000,26040.500000,28056.000000,84.000000,30945.000000
75%,1826.000000,5.002740,59.993018,74.000000,79.000000,27184.000000,28857.000000,85.500000,31506.000000
max,4490.000000,12.301370,147.518430,77.000000,82.000000,28296.000000,30122.000000,98.000000,35849.000000


#### 1. Name of presidents who awarded Bharat Ratna ?

In [ ]:
df.loc[df['Civilian Awards & Honours'].str.contains('Bharat Ratna',na=False),:]

,Name,sex,dob,dod,Service_start,Service_End,Election,Party,Religion,Education,...,Service_End,Num_Service_Days,Num_Service_years,Num_Ser_Months,Service_start_Age,retairment_Age,Service_start_Age_day,retairment_Age_day,age_at_death,age_at_death_days
0,Dr. Rajendra Prasad,M,1884-03-12,1963-02-28,1950-01-26,1962-05-13,"[1952, 1957]",INC,Hinduism,"[M.A , LL.D]",...,1962-05-13,4490,12.301370,147.518430,65,78,24060,28550,78,28841
1,Dr. Sarvepalli Radhakrishnan,M,1888-05-09,1975-04-17,1962-05-13,1967-05-13,1962,Independent,Hinduism,"[M.A, D. Litt. (Hony.), LL.D., D.C.L, Litt. D....",...,1967-05-13,1826,5.002740,59.993018,74,79,27031,28857,86,31753
2,Dr. Zakir Husain,M,1897-08-02,1969-05-03,1967-05-13,1969-05-03,1967,Independent,Islam,"[M.A, Ph.D]",...,1969-05-03,721,1.975342,23.688371,69,71,25485,26206,71,26206
3,Shri Varahagiri Venkata Giri(acting),M,1894-10-08,1980-06-24,1969-05-03,1969-07-20,NaN,Acting_President,Hinduism,"[LL.D, Barrister-at-Law, D. Litt]",...,1969-07-20,78,0.213699,2.562681,74,74,27235,27313,85,31305
5,Shri Varahagiri Venkata Giri,M,1894-10-08,1980-06-24,1969-08-24,1974-08-24,1969,Independent,Hinduism,"[LL.D, Barrister-at-Law, D. Litt]",...,1974-08-24,1826,5.002740,59.993018,74,79,27348,29174,85,31305
13,DR. A.P.J. Abdul Kalam,M,1931-10-15,2015-07-27,2002-07-25,2007-07-25,2002,Independent,Islam,"[M.E , B.Sc]",...,2007-07-25,1826,5.002740,59.993018,70,75,25851,27677,83,30601
15,Shri Pranab Mukherjee,M,1935-12-11,2020-08-31,2012-07-25,2017-07-25,2012,INC,Hinduism,"[M.A, LL.B]",...,2017-07-25,1826,5.002740,59.993018,76,81,27986,29812,84,30945


#### 2. Who is the youngest and aged president in India

In [ ]:
### Youngest full time president of India
df.sort_values('Service_start_Age_day')[df.Party != 'Acting_President'].head(1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,Name,sex,dob,dod,Service_start,Service_End,Election,Party,Religion,Education,...,Service_End,Num_Service_Days,Num_Service_years,Num_Ser_Months,Service_start_Age,retairment_Age,Service_start_Age_day,retairment_Age_day,age_at_death,age_at_death_days
17,Smt Droupadi Murmu,F,1958-06-20,NaT,2022-07-25,NaT,2022,BJP,Hinduism,[BA],...,NaT,<NA>,NaN,NaN,64,<NA>,23411,<NA>,<NA>,<NA>


In [ ]:
### Oldest full time president of India
df.loc[(df.Service_start_Age_day == np.max(df.Service_start_Age_day)) & (df.Party != 'Acting_President')]

,Name,sex,dob,dod,Service_start,Service_End,Election,Party,Religion,Education,...,Service_End,Num_Service_Days,Num_Service_years,Num_Ser_Months,Service_start_Age,retairment_Age,Service_start_Age_day,retairment_Age_day,age_at_death,age_at_death_days
12,Shri K. R. Narayanan,M,1920-02-04,2005-11-09,1997-07-25,2002-07-25,1997,INC,Hinduism,"[M.A, B.Sc]",...,2002-07-25,1826,5.00274,59.993018,77,82,28296,30122,85,31325


#### 3. Details of full time president who was in the service for long time

In [ ]:
df.loc[(df.Num_Service_years == np.max(df.Num_Service_years)) & (df.Party != 'Acting_President')]

,Name,sex,dob,dod,Service_start,Service_End,Election,Party,Religion,Education,...,Service_End,Num_Service_Days,Num_Service_years,Num_Ser_Months,Service_start_Age,retairment_Age,Service_start_Age_day,retairment_Age_day,age_at_death,age_at_death_days
0,Dr. Rajendra Prasad,M,1884-03-12,1963-02-28,1950-01-26,1962-05-13,"[1952, 1957]",INC,Hinduism,"[M.A , LL.D]",...,1962-05-13,4490,12.30137,147.51843,65,78,24060,28550,78,28841


#### 4. Name of presidnt who died during his office ?

In [ ]:
df.loc[df.Service_End == df.dod]

,Name,sex,dob,dod,Service_start,Service_End,Election,Party,Religion,Education,...,Service_End,Num_Service_Days,Num_Service_years,Num_Ser_Months,Service_start_Age,retairment_Age,Service_start_Age_day,retairment_Age_day,age_at_death,age_at_death_days
2,Dr. Zakir Husain,M,1897-08-02,1969-05-03,1967-05-13,1969-05-03,1967,Independent,Islam,"[M.A, Ph.D]",...,1969-05-03,721,1.975342,23.688371,69,71,25485,26206,71,26206
6,Dr. Fakhruddin Ali Ahmed,M,1905-05-13,1977-02-11,1974-08-24,1977-02-11,1974,INC,Islam,"[M.D, practiced Law]",...,1977-02-11,902,2.471233,29.635105,69,71,25305,26207,71,26207


In [ ]:
### saving new file
#  df.to_csv('updated_data_1.csv')
#  from google.colab import files
#  files.download("updated_data_1.csv")